## 多个 feas_iti 且最后一趟列车 id 相同的乘客

|           |     rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
| :-------- | ------: | -----: | ---------: | -----: | -------: | -------: | --------: |
| 93194730  | 1525794 |      2 | 1076103701 |      2 | 10301630 |    65071 |     65670 |
| 8610761   |  119014 |      6 | 1101102701 |      3 | 11002833 |    30045 |     30791 |
| 18578209  |  230428 |      6 | 1053108503 |      2 | 10401849 |    29715 |     30255 |
| 72855273  | 1321497 |      3 | 1110112802 |      1 | 10401844 |    60803 |     61537 |
| 105488240 | 1781873 |      2 | 1028106602 |      2 | 10401738 |    69396 |     69625 |
| 90908613  | 1485498 |      2 | 1023112302 |      2 | 10702370 |    64374 |     64780 |
| 91388272  | 1494286 |      1 | 1104111501 |      2 | 10702247 |    65070 |     65278 |
| 53621559  |  957712 |      6 | 1121102503 |      3 | 10401786 |    49174 |     50141 |
| 19767473  |  243641 |      1 | 1081103901 |      2 | 10702323 |    29509 |     31141 |
| 107978876 | 1880069 |      2 | 1129107002 |      1 | 10201226 |    73278 |     74154 |
| 106615427 | 1817923 |      1 | 1027110801 |      2 | 10301362 |    71471 |     73514 |
| 76737269  | 1359494 |      1 | 1099102101 |      2 | 10301421 |    64071 |     65266 |
| 100422001 | 1661221 |      5 | 1061108702 |      1 | 10100270 |    66558 |     66886 |
| 2408620   |   41409 |      2 | 1111106502 |      1 | 10401886 |    25727 |     25836 |
| 101105912 | 1675451 |      1 | 1110101301 |      1 | 10401914 |    66944 |     67031 |
| 92513072  | 1514419 |      3 | 1097110302 |      3 | 10200839 |    65302 |     65408 |
| 81121188  | 1412555 |      1 | 1061109504 |      2 | 10201199 |    63371 |     63480 |
| 108069499 | 1886032 |      1 | 1032108201 |      1 | 10200950 |    73568 |     75192 |
| 73737366  | 1331466 |      1 | 1086110901 |      2 | 10702414 |    61616 |     62107 |
| 48046191  |  811513 |      3 | 1076103602 |      3 | 10301265 |    42747 |     43084 |


In [1]:
from src import config
from src.utils import *
from src.globals import *

config.load_config()

left = read_(config.CONFIG["results"]["left"], show_timer=False)
print(left.shape)
print(left.drop_duplicates("rid").shape)
print(left.drop_duplicates(["rid", "iti_id"]).shape)
left

[INFO] Reading file: results\trajectory\left.pkl
(46637884, 7)
(1195446, 7)
(16339032, 7)


,rid,iti_id,path_id,seg_id,train_id,board_ts,alight_ts
2,285,1,1120101001,1,10702170,21132,22747
3,285,2,1120101001,1,10702170,25273,26962
4,285,3,1120101001,1,10702186,21797,23422
5,285,4,1120101001,1,10702186,25949,27655
6,285,5,1120101001,1,10702207,22652,24277
...,...,...,...,...,...,...,...
109949952,2046865,2,1097101501,1,10100645,85287,85823
109949955,2046868,1,1097101501,1,10100421,84945,85481
109949956,2046868,2,1097101501,1,10100645,85287,85823
109949965,2046883,1,1129101001,1,10702289,85531,85741


In [12]:
# Extract path segments with transfer
from src.walk_time_dis_calculator import WalkTimeDisModel
from src.walk_time_filter import get_path_seg_to_pp_ids, get_transfer_from_feas_iti

wtdc = WalkTimeDisModel(etd=get_etd(), ttd=get_ttd())
left = read_(config.CONFIG["results"]["left"], show_timer=False)


df = get_transfer_from_feas_iti(df_feas_iti=left)  # ['rid', 'iti_id', 'path_id', 'seg_id', 'alight_ts', 'transfer_time']

df_ps2pp = get_path_seg_to_pp_ids()  # ["path_id", "seg_id", "pp_id1", "pp_id2", "transfer_type"]
df_ps2pp["pp_id_min"] = df_ps2pp[["pp_id1", "pp_id2"]].min(axis=1)
df_ps2pp["pp_id_max"] = df_ps2pp[["pp_id1", "pp_id2"]].max(axis=1)
df_ps2pp.drop(columns=["pp_id1", "pp_id2"], inplace=True)

df = df.merge(df_ps2pp, on=["path_id", "seg_id"], how="left")

df

[INFO] Reading file: results\trajectory\left.pkl


,rid,path_id,seg_id,alight_ts,transfer_time,transfer_type,pp_id_min,pp_id_max
0,285,1120101002,1,21238,2172.0,egress-entry,103801,103802
1,285,1120101002,2,24412,2860.0,egress-entry,102201,102202
2,285,1120101002,1,21238,2172.0,egress-entry,103801,103802
3,285,1120101002,2,24412,108.0,egress-entry,102201,102202
4,285,1120101002,1,21238,2172.0,egress-entry,103801,103802
...,...,...,...,...,...,...,...,...
30298847,2046611,1017105703,1,84511,84.0,egress-entry,109701,109702
30298848,2046611,1017105703,2,85547,370.0,egress-entry,105201,105202
30298849,2046611,1017105704,1,84642,197.0,egress-entry,101601,101602
30298850,2046611,1017105704,2,85041,61.0,egress-entry,102201,102202


In [17]:
df["dis"] = 0.0  # to be filled
for (pp_id_min, pp_id_max), df_ in df.groupby(["pp_id_min", "pp_id_max"]):
    print(pp_id_min, pp_id_max, df_.shape)
    dis = wtdc.compute_transfer_cdf_from_pp(
        pp_id_min, pp_id_max, 
        times_start=np.zeros(df_.shape[0]), 
        times_end=df_["transfer_time"].values
        )
    df.loc[df_.index, "dis"] = dis
df

101001 101002 (806749, 9)
101601 101602 (3305314, 9)
102201 102202 (2077578, 9)
102401 102402 (2356933, 9)
102801 102802 (3698164, 9)
103201 103201 (1171607, 9)
103201 103202 (362995, 9)
103202 103202 (984515, 9)
103301 103302 (1281002, 9)
103301 103303 (1107323, 9)
103801 103802 (740332, 9)
103801 103803 (357249, 9)
103801 103804 (357258, 9)
103802 103802 (148790, 9)
103803 103804 (323337, 9)
104301 104302 (4093, 9)
105201 105202 (1098445, 9)
106101 106102 (1169616, 9)
106301 106302 (3135147, 9)
109701 109702 (2947971, 9)
109801 109801 (3065, 9)
112701 112702 (1826758, 9)
112901 112902 (1034611, 9)


,rid,path_id,seg_id,alight_ts,transfer_time,transfer_type,pp_id_min,pp_id_max,dis
0,285,1120101002,1,21238,2172.0,egress-entry,103801,103802,1.000000
1,285,1120101002,2,24412,2860.0,egress-entry,102201,102202,1.000000
2,285,1120101002,1,21238,2172.0,egress-entry,103801,103802,1.000000
3,285,1120101002,2,24412,108.0,egress-entry,102201,102202,0.178590
4,285,1120101002,1,21238,2172.0,egress-entry,103801,103802,1.000000
...,...,...,...,...,...,...,...,...,...
30298847,2046611,1017105703,1,84511,84.0,egress-entry,109701,109702,0.259494
30298848,2046611,1017105703,2,85547,370.0,egress-entry,105201,105202,0.975816
30298849,2046611,1017105704,1,84642,197.0,egress-entry,101601,101602,0.689612
30298850,2046611,1017105704,2,85041,61.0,egress-entry,102201,102202,0.046804


In [24]:
# df['dis'].describe()
# random examine

# df.sample(n=20)
for row in df.sample(n=20).itertuples():
    print(row)
    dis = wtdc.compute_transfer_cdf(path_id=row.path_id, seg_id=row.seg_id, times_start=0, times_end=row.transfer_time)
    print(dis == row.dis)

Pandas(Index=4118315, rid=180473, path_id=1031104207, seg_id=1, alight_ts=29553, transfer_time=273.0, transfer_type='egress-entry', pp_id_min=103201, pp_id_max=103202, dis=0.8897528253882264)
True
Pandas(Index=6352997, rid=253631, path_id=1106102302, seg_id=3, alight_ts=31598, transfer_time=627.0, transfer_type='egress-entry', pp_id_min=109701, pp_id_max=109702, dis=1.0)
True
Pandas(Index=29681441, rid=1930825, path_id=1029109301, seg_id=1, alight_ts=75385, transfer_time=206.0, transfer_type='egress-entry', pp_id_min=106301, pp_id_max=106302, dis=0.7426602819878414)
True
Pandas(Index=5986974, rid=240945, path_id=1077108701, seg_id=2, alight_ts=31793, transfer_time=680.0, transfer_type='egress-entry', pp_id_min=102401, pp_id_max=102402, dis=1.0)
True
Pandas(Index=1119388, rid=81170, path_id=1130104201, seg_id=1, alight_ts=28404, transfer_time=45.0, transfer_type='egress-entry', pp_id_min=112901, pp_id_max=112902, dis=0.011019986143771)
True
Pandas(Index=13737453, rid=717210, path_id=110